![Title](./image/title.png)

## Problem Scenario: Controlling Access and Improving Performance

The Proxy pattern addresses two key problems:

1.  **Controlling Access:**
    *   Need to restrict access to certain resources (e.g., websites) based on certain conditions (e.g., user, time of day).
    *   Don't want to modify the original object (e.g., the `Internet` class) to add access control logic, as it would affect all users.
    *   Solution: An intermediary ("proxy") that intercepts requests and applies access rules before forwarding them to the original object.

2.  **Improving Performance (Caching):**
    *   Certain operations are expensive and time-consuming (e.g., downloading videos).
    *   Repeatedly performing the same operation is inefficient.
    *   Solution: A proxy that caches the results of expensive operations.  Subsequent requests for the same result are served from the cache, avoiding the need to re-execute the operation.

**Essentially:**

The Proxy pattern lets you add behavior before or after a request reaches the original object, without modifying the original object itself. This behavior can be access control, caching, or other types of enhancements.

## Problem: Restricting Website Access

The goal is to restrict access to specific websites (e.g., "banned.com") for certain users, without affecting other users or modifying the core `Internet` class. Directly modifying the `Internet` class would enforce the restriction globally.

In [ ]:
class Internet:
    def connect_to(self, host):
        """Connects to a given host."""
        pass

class RealInternet(Internet):
    def connect_to(self, host):
        """Connects to a given host, with a ban."""
        if host == "banned.com":
            print("Access Denied!")
            return

        print(f"Opened connection to {host}")

internet = RealInternet()
internet.connect_to("google.com")
internet.connect_to("banned.com")  # Output: Access Denied!

## Solution: ProxyInternet for Selective Banning

The `ProxyInternet` class implements the `Internet` interface and intercepts calls to `connect_to()`. It checks if the requested host is in the `banned_sites` list. If it is, access is denied; otherwise, the connection is forwarded to the `RealInternet` object. This allows us to selectively apply the banning functionality without modifying `RealInternet` and without affecting other clients that might not want to use the proxy.

In [ ]:
class Internet:
    def connect_to(self, host):
        """Connects to a given host."""
        pass


class RealInternet(Internet):
    def connect_to(self, host):
        """Connects to a given host."""
        print(f"Opened connection to {host}")


class ProxyInternet(Internet):
    def __init__(self):
        self.real_internet = RealInternet()
        self.banned_sites = ["banned.com"]

    def connect_to(self, host):
        """Connects to a given host, checking for banned sites first."""
        if host in self.banned_sites:
            print("Access Denied!")
        else:
            self.real_internet.connect_to(host)


internet = ProxyInternet()
internet.connect_to("google.com")
internet.connect_to("banned.com")

real_internet = RealInternet()
real_internet.connect_to("banned.com")

## Scenario: Inefficient Video Downloading (Before Proxy)

The `RealVideoDownloader` retrieves video metadata every time `getVideo` is called, even for videos that have already been downloaded.  This is inefficient and wastes resources.

In [ ]:
class Video:
    def __init__(self, name):
        self.name = name

class VideoDownloader:
    def getVideo(self, videoName):
        pass

class RealVideoDownloader(VideoDownloader):
    def getVideo(self, videoName):
        print("Connecting to https://www.youtube.com/")
        print("Downloading Video")
        print("Retrieving Video Metadata")
        return Video(videoName)

downloader = RealVideoDownloader()

video1 = downloader.getVideo("geekific") # Connects, downloads, retrieves metadata
video2 = downloader.getVideo("geekific") # Connects, downloads, retrieves metadata AGAIN!

video3 = downloader.getVideo("likeNsub")
video4 = downloader.getVideo("likeNsub")

video5 = downloader.getVideo("geekific") # Connects, downloads, retrieves metadata AGAIN!

## Solution: ProxyVideoDownloader for Caching

The `ProxyVideoDownloader` implements the `VideoDownloader` interface and caches the downloaded videos.  When `getVideo` is called, it first checks if the video is in the cache. If it is, the cached video is returned. Otherwise, the `RealVideoDownloader` is used to download the video, and the result is stored in the cache before being returned. This significantly reduces the number of times the `RealVideoDownloader` is called.

![Example](./image/example.png)

In [ ]:
class Video:
    def __init__(self, name):
        self.name = name

class VideoDownloader:
    def getVideo(self, videoName):
        pass

class RealVideoDownloader(VideoDownloader):
    def getVideo(self, videoName):
        print("Connecting to https://www.youtube.com/")
        print("Downloading Video")
        print("Retrieving Video Metadata")
        return Video(videoName) 

class ProxyVideoDownloader(VideoDownloader):
    def __init__(self):
        self.real_downloader = RealVideoDownloader()
        self.video_cache = {}
        
    def getVideo(self, videoName):
        if videoName not in self.video_cache:
            print(f"(Proxy) Downloading {videoName} from the source.")
            self.video_cache[videoName] = self.real_downloader.getVideo(videoName)
        else:
            print(f"(Proxy) Retrieving {videoName} from cache.")
        return self.video_cache[videoName]

downloader = ProxyVideoDownloader()

video1 = downloader.getVideo("geekific") # Connects, downloads, retrieves metadata
video2 = downloader.getVideo("geekific") # Retrieves from cache

video3 = downloader.getVideo("likeNsub") # Connects, downloads, retrieves metadata

video4 = downloader.getVideo("likeNsub") # Retrieves from cache
video5 = downloader.getVideo("geekific") # Retrieves from cache